In [1]:
import numpy as np

In [2]:
mass = [482.01, 108.14, 279.92, 80.91]  # core, linker, ext, bond(HBr)

def renorm(goal, mass):
    """
    Only works for situation like tri-core + dual-linker + dual-ext
    a1*n1 + a2*n2 + a3*n3 = a4; n1, n2, n3 should be positive integer, they are # of cores, ext and free linker, respectively.
    """
    a = [mass[0]+mass[1]-2*mass[3], mass[2]+mass[1]-2*mass[3], mass[1]-mass[3], goal+mass[1]-2*mass[3]]
    return a

def get_comb(a):
    N = [a[3]//a[0], a[3]//a[1], a[3]//a[2]]
    X = np.mgrid[:N[0]+1,:N[1]+1,:N[2]+1]
    return X

def unit_calc(X, a):
    x1, x2, x3 = X
    a1, a2, a3 = a[:-1]
    return a1*x1 + a2*x2 + a3*x3

def get_mass(res, mass):
    return res + 2*mass[3] - mass[1]   

def get_mask(X):
    x1, x2, x3 = X
    return x3 <= x1+2

def find_closest(res, a, k=5, mask=None):
    """return first k smallest err index and its results and its err (id, res, err)"""
    if mask is None:
        err = np.abs(res - a[-1])
    else:
        err = np.where(mask, np.abs(res - a[-1]), np.inf)
    idx = tuple(np.array(np.unravel_index(np.argsort(err, axis=None), err.shape))[:,:k])
    return idx, err[idx]

def get_fraction(idx):
    x1, x2, x3 = idx
    n1 = x1  # cores
    n3 = x2  # ext
    nf = x3  # free linker
    nb = n1 + n3 -1  # bonded linker
    n2 = nf + nb  # linkers
    n4 = nf + 2*nb  # HBr
    return n1, n2, n3, n4


In [7]:
def find_solution(goal, mass=mass, k=1):
    a = renorm(goal, mass)
    X = get_comb(a)
    summation = unit_calc(X, a)
    mask = get_mask(X)
    idx, err = find_closest(summation, a, k=k, mask=mask)    
    total_mass = get_mass(summation, mass)
    attempted_mass = np.sort(np.ravel(total_mass)[np.ravel(mask)])
    print("all attempted valid mass:")
    print(attempted_mass)
    return idx, total_mass[idx], total_mass[idx]-goal

In [11]:
idx, calc_mass, err = find_solution(1223.724, k=3)

print("index:", np.array(idx).T)
print("calculated mass:", calc_mass)
print("error to goal:", err)

all attempted valid mass:
[  53.68   80.91  108.14  279.92  307.15  334.38  482.01  506.16  509.24
  533.39  536.47  560.62  563.7   708.25  732.4   735.48  759.63  762.71
  786.86  789.94  910.34  934.49  937.57  958.64  961.72  964.8   985.87
  988.95  992.03 1013.1  1016.18 1019.26 1136.58 1160.73 1163.81 1184.88
 1187.96 1191.04 1212.11 1215.19 1218.27 1239.34 1242.42 1245.5  1362.82
 1386.97 1390.05 1414.2  1417.28 1441.43 1444.51 1468.66 1471.74 1589.06
 1613.21 1616.29 1640.44 1643.52 1667.67 1670.75 1694.9  1697.98 1815.3
 1842.53 1869.76 1896.99 1924.22 2041.54 2068.77 2096.   2123.23 2150.46]
index: [[2 1 3]
 [1 3 2]
 [0 5 1]]
calculated mass: [1218.27 1215.19 1212.11]
error to goal: [ -5.454  -8.534 -11.614]


In [5]:
frac = np.array(get_fraction(idx)).T
print("[cores linkers exts sites]:\n", frac)

[cores linkers exts sites]:
 [[1 4 4 8]
 [2 4 2 7]
 [3 4 0 6]]
